# **ortools 라이브러리를 활용하여 최적화하자**

문제3 운송사 최대배정 제한 제약조건을 반영한 '리스크 분산을 위한 운송사 선정 최적화'와 변형문제를 ortools라이브러리를 이용하여 구함

(동일 프로그램에서 입출력 파일명만 변경)

1) 3개운송사_할당제약_기준정보

2) 3개운송사_3개운송구간_기준정보

3) 4개운송사_3개운송구간_기준정보

4) 1개운송사만선정_기준정보

In [1]:
!pip install ortools

3개운송사_할당제약_기준정보

In [ ]:
import pandas as pd
import time
import math
from ortools.linear_solver import pywraplp

def optimize_truck_assignment():
    start_time = time.time()

    #--------------------------------------------------------
    # 1) 데이터 불러오기
    #--------------------------------------------------------
    input_file = "/content/sample_data/3개운송사_할당제약_기준정보.xlsx"
    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']
    max_trucker_num = int(pd.read_excel(input_file, sheet_name=0, usecols="M").iloc[0, 0])
    max_ratio = float(pd.read_excel(input_file, sheet_name=0, usecols="P").iloc[0, 0])

    # 결측값 제거 및 숫자 변환
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)

    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df_demand.dropna(subset=['필요대수'], inplace=True)

    #--------------------------------------------------------
    # 2) 파라미터 세팅
    #--------------------------------------------------------
    cost_dict = {}      # (i, r, t): 운송단가
    capacity_dict = {}  # (i, r, t): 트럭보유대수
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])
        cost_dict[key] = row['운송단가']
        capacity_dict[key] = row['트럭대수']

    demand_dict = {}    # (r, t): 필요대수
    for _, row in df_demand.iterrows():
        key = (row['운송구간'], row['트럭유형'])
        demand_dict[key] = row['필요대수']

    truckers = df['운송사'].unique()
    route_type_list = demand_dict.keys()

    #--------------------------------------------------------
    # 3) OR-Tools 모델 생성
    #--------------------------------------------------------
    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print("Solver 초기화 실패")
        return

    #--------------------------------------------------------
    # 4) 변수 정의
    #--------------------------------------------------------
    x_vars = {}  # 트럭 수 변수
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = solver.IntVar(0, capacity_dict[(i, r, t)], f"x_{i}_{r}_{t}")

    y_vars = {i: solver.BoolVar(f"y_{i}") for i in truckers}

    #--------------------------------------------------------
    # 5) 목적함수: 총 운송비 최소화
    #--------------------------------------------------------
    solver.Minimize(solver.Sum(cost_dict[(i, r, t)] * x_vars[(i, r, t)] for (i, r, t) in x_vars))

    #--------------------------------------------------------
    # 6) 제약조건 추가
    #--------------------------------------------------------

    # (1) 수요 충족
    for (r, t) in route_type_list:
        solver.Add(solver.Sum(x_vars[(i, r, t)] for i in truckers if (i, r, t) in x_vars) == demand_dict[(r, t)])

    # (2) 트럭 보유량 이내로 배정
    for (i, r, t), cap in capacity_dict.items():
        solver.Add(x_vars[(i, r, t)] <= cap)

    # (3) 운송사 선택 여부에 따라 트럭 배정 가능
    for (i, r, t) in x_vars:
        solver.Add(x_vars[(i, r, t)] <= capacity_dict[(i, r, t)] * y_vars[i])

    # (4) 선택된 운송사 수 제한
    solver.Add(solver.Sum(y_vars[i] for i in truckers) <= max_trucker_num)

    # (5) 운송사당 최대 배정량 제한 (X_MAX 적용)
    for (r, t) in route_type_list:
        demand = demand_dict[(r, t)]
        x_max = math.ceil(demand * max_ratio)
        for i in truckers:
            if (i, r, t) in x_vars:
                solver.Add(x_vars[(i, r, t)] <= x_max)

    #--------------------------------------------------------
    # 7) 최적화 실행
    #--------------------------------------------------------
    status = solver.Solve()

    if status not in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print("=> 유효한 해를 찾지 못했습니다.")
        return

    total_cost = solver.Objective().Value()
    print("총 운송비:", total_cost)

    #--------------------------------------------------------
    # 8) 결과 저장
    #--------------------------------------------------------
    result_data = []
    for (i, r, t), var in x_vars.items():
        val = var.solution_value()
        if val > 0:
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(val)])

    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    df_result.sort_values(by=['운송구간', '트럭유형'], inplace=True)

    output_file = "/content/sample_data/3개운송사_할당제약_최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))

    end_time = time.time()
    print("\n프로그램 수행시간(초):", round(end_time - start_time, 4))

if __name__ == "__main__":
    optimize_truck_assignment()


총 운송비: 7800.0

--- 배정 결과 ---
  운송사 운송구간 트럭유형  운송단가  선택된트럭수
운송사01 구간01 트럭01   100       8
운송사03 구간01 트럭01   200       2
운송사01 구간01 트럭02   100       8
운송사03 구간01 트럭02   200       2
운송사01 구간02 트럭01   100       8
운송사03 구간02 트럭01   200       2
운송사01 구간02 트럭02   500       2
운송사03 구간02 트럭02   400       8

프로그램 수행시간(초): 0.7352


# **변형된 문제를 가지고 결괏값을 검증하자**

3개운송사-3개운송구간_기준정보

In [ ]:
import pandas as pd
import time
import math
from ortools.linear_solver import pywraplp

def optimize_truck_assignment():
    start_time = time.time()

    #--------------------------------------------------------
    # 1) 데이터 불러오기
    #--------------------------------------------------------
    input_file = "/content/sample_data/3개운송사_3개운송구간_기준정보.xlsx"
    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']
    max_trucker_num = int(pd.read_excel(input_file, sheet_name=0, usecols="M").iloc[0, 0])
    max_ratio = float(pd.read_excel(input_file, sheet_name=0, usecols="P").iloc[0, 0])

    # 결측값 제거 및 숫자 변환
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)

    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df_demand.dropna(subset=['필요대수'], inplace=True)

    #--------------------------------------------------------
    # 2) 파라미터 세팅
    #--------------------------------------------------------
    cost_dict = {}      # (i, r, t): 운송단가
    capacity_dict = {}  # (i, r, t): 트럭보유대수
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])
        cost_dict[key] = row['운송단가']
        capacity_dict[key] = row['트럭대수']

    demand_dict = {}    # (r, t): 필요대수
    for _, row in df_demand.iterrows():
        key = (row['운송구간'], row['트럭유형'])
        demand_dict[key] = row['필요대수']

    truckers = df['운송사'].unique()
    route_type_list = demand_dict.keys()

    #--------------------------------------------------------
    # 3) OR-Tools 모델 생성
    #--------------------------------------------------------
    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print("Solver 초기화 실패")
        return

    #--------------------------------------------------------
    # 4) 변수 정의
    #--------------------------------------------------------
    x_vars = {}  # 트럭 수 변수
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = solver.IntVar(0, capacity_dict[(i, r, t)], f"x_{i}_{r}_{t}")

    y_vars = {i: solver.BoolVar(f"y_{i}") for i in truckers}

    #--------------------------------------------------------
    # 5) 목적함수: 총 운송비 최소화
    #--------------------------------------------------------
    solver.Minimize(solver.Sum(cost_dict[(i, r, t)] * x_vars[(i, r, t)] for (i, r, t) in x_vars))

    #--------------------------------------------------------
    # 6) 제약조건 추가
    #--------------------------------------------------------

    # (1) 수요 충족
    for (r, t) in route_type_list:
        solver.Add(solver.Sum(x_vars[(i, r, t)] for i in truckers if (i, r, t) in x_vars) == demand_dict[(r, t)])

    # (2) 트럭 보유량 이내로 배정
    for (i, r, t), cap in capacity_dict.items():
        solver.Add(x_vars[(i, r, t)] <= cap)

    # (3) 운송사 선택 여부에 따라 트럭 배정 가능
    for (i, r, t) in x_vars:
        solver.Add(x_vars[(i, r, t)] <= capacity_dict[(i, r, t)] * y_vars[i])

    # (4) 선택된 운송사 수 제한
    solver.Add(solver.Sum(y_vars[i] for i in truckers) <= max_trucker_num)

    # (5) 운송사당 최대 배정량 제한 (X_MAX 적용)
    for (r, t) in route_type_list:
        demand = demand_dict[(r, t)]
        x_max = math.ceil(demand * max_ratio)
        for i in truckers:
            if (i, r, t) in x_vars:
                solver.Add(x_vars[(i, r, t)] <= x_max)

    #--------------------------------------------------------
    # 7) 최적화 실행
    #--------------------------------------------------------
    status = solver.Solve()

    if status not in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print("=> 유효한 해를 찾지 못했습니다.")
        return

    total_cost = solver.Objective().Value()
    print("총 운송비:", total_cost)

    #--------------------------------------------------------
    # 8) 결과 저장
    #--------------------------------------------------------
    result_data = []
    for (i, r, t), var in x_vars.items():
        val = var.solution_value()
        if val > 0:
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(val)])

    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    df_result.sort_values(by=['운송구간', '트럭유형'], inplace=True)

    output_file = "/content/sample_data/3개운송사_3개운송구간_최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))

    end_time = time.time()
    print("\n프로그램 수행시간(초):", round(end_time - start_time, 4))

if __name__ == "__main__":
    optimize_truck_assignment()


총 운송비: 32200.0

--- 배정 결과 ---
  운송사 운송구간 트럭유형  운송단가  선택된트럭수
운송사02 구간01 트럭01   150       8
운송사03 구간01 트럭01   200       2
운송사02 구간01 트럭02   150       8
운송사03 구간01 트럭02   200       2
운송사02 구간02 트럭01   150       8
운송사03 구간02 트럭01   200       2
운송사02 구간02 트럭02 10000       2
운송사03 구간02 트럭02   400       8
운송사02 구간03 트럭01   400       8
운송사03 구간03 트럭01   500       2

프로그램 수행시간(초): 0.0695


4개운송사_3개운송구간_기준정보

In [ ]:
import pandas as pd
import time
import math
from ortools.linear_solver import pywraplp

def optimize_truck_assignment():
    start_time = time.time()

    #--------------------------------------------------------
    # 1) 데이터 불러오기
    #--------------------------------------------------------
    input_file = "/content/sample_data/4개운송사_3개운송구간_기준정보.xlsx"
    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']
    max_trucker_num = int(pd.read_excel(input_file, sheet_name=0, usecols="M").iloc[0, 0])
    max_ratio = float(pd.read_excel(input_file, sheet_name=0, usecols="P").iloc[0, 0])

    # 결측값 제거 및 숫자 변환
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)

    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df_demand.dropna(subset=['필요대수'], inplace=True)

    #--------------------------------------------------------
    # 2) 파라미터 세팅
    #--------------------------------------------------------
    cost_dict = {}      # (i, r, t): 운송단가
    capacity_dict = {}  # (i, r, t): 트럭보유대수
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])
        cost_dict[key] = row['운송단가']
        capacity_dict[key] = row['트럭대수']

    demand_dict = {}    # (r, t): 필요대수
    for _, row in df_demand.iterrows():
        key = (row['운송구간'], row['트럭유형'])
        demand_dict[key] = row['필요대수']

    truckers = df['운송사'].unique()
    route_type_list = demand_dict.keys()

    #--------------------------------------------------------
    # 3) OR-Tools 모델 생성
    #--------------------------------------------------------
    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print("Solver 초기화 실패")
        return

    #--------------------------------------------------------
    # 4) 변수 정의
    #--------------------------------------------------------
    x_vars = {}  # 트럭 수 변수
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = solver.IntVar(0, capacity_dict[(i, r, t)], f"x_{i}_{r}_{t}")

    y_vars = {i: solver.BoolVar(f"y_{i}") for i in truckers}

    #--------------------------------------------------------
    # 5) 목적함수: 총 운송비 최소화
    #--------------------------------------------------------
    solver.Minimize(solver.Sum(cost_dict[(i, r, t)] * x_vars[(i, r, t)] for (i, r, t) in x_vars))

    #--------------------------------------------------------
    # 6) 제약조건 추가
    #--------------------------------------------------------

    # (1) 수요 충족
    for (r, t) in route_type_list:
        solver.Add(solver.Sum(x_vars[(i, r, t)] for i in truckers if (i, r, t) in x_vars) == demand_dict[(r, t)])

    # (2) 트럭 보유량 이내로 배정
    for (i, r, t), cap in capacity_dict.items():
        solver.Add(x_vars[(i, r, t)] <= cap)

    # (3) 운송사 선택 여부에 따라 트럭 배정 가능
    for (i, r, t) in x_vars:
        solver.Add(x_vars[(i, r, t)] <= capacity_dict[(i, r, t)] * y_vars[i])

    # (4) 선택된 운송사 수 제한
    solver.Add(solver.Sum(y_vars[i] for i in truckers) <= max_trucker_num)

    # (5) 운송사당 최대 배정량 제한 (X_MAX 적용)
    for (r, t) in route_type_list:
        demand = demand_dict[(r, t)]
        x_max = math.ceil(demand * max_ratio)
        for i in truckers:
            if (i, r, t) in x_vars:
                solver.Add(x_vars[(i, r, t)] <= x_max)

    #--------------------------------------------------------
    # 7) 최적화 실행
    #--------------------------------------------------------
    status = solver.Solve()

    if status not in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print("=> 유효한 해를 찾지 못했습니다.")
        return

    total_cost = solver.Objective().Value()
    print("총 운송비:", total_cost)

    #--------------------------------------------------------
    # 8) 결과 저장
    #--------------------------------------------------------
    result_data = []
    for (i, r, t), var in x_vars.items():
        val = var.solution_value()
        if val > 0:
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(val)])

    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    df_result.sort_values(by=['운송구간', '트럭유형'], inplace=True)

    output_file = "/content/sample_data/4개운송사_3개운송구간_최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))

    end_time = time.time()
    print("\n프로그램 수행시간(초):", round(end_time - start_time, 4))

if __name__ == "__main__":
    optimize_truck_assignment()


=> 유효한 해를 찾지 못했습니다.


1개운송사만선정_기준정보

In [ ]:
import pandas as pd
import time
import math
from ortools.linear_solver import pywraplp

def optimize_truck_assignment():
    start_time = time.time()

    #--------------------------------------------------------
    # 1) 데이터 불러오기
    #--------------------------------------------------------
    input_file = "/content/sample_data/1개운송사만선정_기준정보.xlsx"
    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']
    max_trucker_num = int(pd.read_excel(input_file, sheet_name=0, usecols="M").iloc[0, 0])
    max_ratio = float(pd.read_excel(input_file, sheet_name=0, usecols="P").iloc[0, 0])

    # 결측값 제거 및 숫자 변환
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)

    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df_demand.dropna(subset=['필요대수'], inplace=True)

    #--------------------------------------------------------
    # 2) 파라미터 세팅
    #--------------------------------------------------------
    cost_dict = {}      # (i, r, t): 운송단가
    capacity_dict = {}  # (i, r, t): 트럭보유대수
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])
        cost_dict[key] = row['운송단가']
        capacity_dict[key] = row['트럭대수']

    demand_dict = {}    # (r, t): 필요대수
    for _, row in df_demand.iterrows():
        key = (row['운송구간'], row['트럭유형'])
        demand_dict[key] = row['필요대수']

    truckers = df['운송사'].unique()
    route_type_list = demand_dict.keys()

    #--------------------------------------------------------
    # 3) OR-Tools 모델 생성
    #--------------------------------------------------------
    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print("Solver 초기화 실패")
        return

    #--------------------------------------------------------
    # 4) 변수 정의
    #--------------------------------------------------------
    x_vars = {}  # 트럭 수 변수
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = solver.IntVar(0, capacity_dict[(i, r, t)], f"x_{i}_{r}_{t}")

    y_vars = {i: solver.BoolVar(f"y_{i}") for i in truckers}

    #--------------------------------------------------------
    # 5) 목적함수: 총 운송비 최소화
    #--------------------------------------------------------
    solver.Minimize(solver.Sum(cost_dict[(i, r, t)] * x_vars[(i, r, t)] for (i, r, t) in x_vars))

    #--------------------------------------------------------
    # 6) 제약조건 추가
    #--------------------------------------------------------

    # (1) 수요 충족
    for (r, t) in route_type_list:
        solver.Add(solver.Sum(x_vars[(i, r, t)] for i in truckers if (i, r, t) in x_vars) == demand_dict[(r, t)])

    # (2) 트럭 보유량 이내로 배정
    for (i, r, t), cap in capacity_dict.items():
        solver.Add(x_vars[(i, r, t)] <= cap)

    # (3) 운송사 선택 여부에 따라 트럭 배정 가능
    for (i, r, t) in x_vars:
        solver.Add(x_vars[(i, r, t)] <= capacity_dict[(i, r, t)] * y_vars[i])

    # (4) 선택된 운송사 수 제한
    solver.Add(solver.Sum(y_vars[i] for i in truckers) <= max_trucker_num)

    # (5) 운송사당 최대 배정량 제한 (X_MAX 적용)
    for (r, t) in route_type_list:
        demand = demand_dict[(r, t)]
        x_max = math.ceil(demand * max_ratio)
        for i in truckers:
            if (i, r, t) in x_vars:
                solver.Add(x_vars[(i, r, t)] <= x_max)

    #--------------------------------------------------------
    # 7) 최적화 실행
    #--------------------------------------------------------
    status = solver.Solve()

    if status not in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print("=> 유효한 해를 찾지 못했습니다.")
        return

    total_cost = solver.Objective().Value()
    print("총 운송비:", total_cost)

    #--------------------------------------------------------
    # 8) 결과 저장
    #--------------------------------------------------------
    result_data = []
    for (i, r, t), var in x_vars.items():
        val = var.solution_value()
        if val > 0:
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(val)])

    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    df_result.sort_values(by=['운송구간', '트럭유형'], inplace=True)

    output_file = "/content/sample_data/1개운송사만선정_최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))

    end_time = time.time()
    print("\n프로그램 수행시간(초):", round(end_time - start_time, 4))

if __name__ == "__main__":
    optimize_truck_assignment()


총 운송비: 108500.0

--- 배정 결과 ---
  운송사 운송구간 트럭유형  운송단가  선택된트럭수
운송사02 구간01 트럭01   150      10
운송사02 구간01 트럭02   150      10
운송사02 구간02 트럭01   150      10
운송사02 구간02 트럭02 10000      10
운송사02 구간03 트럭01   400      10

프로그램 수행시간(초): 0.0735


# **큰 크기의 문제를 풀어보자**
- 큰 크기의 문제를 풀기전 사전 정제된 데이터로 테스트(100개 운송사 1차 전처리 후 데이터)
- 100개운송사_50개운송구간_기준정보A선행결과통합.xlsx 에 대하여 약 28분 소요



In [2]:
import pandas as pd
import time
import math
from ortools.linear_solver import pywraplp

def optimize_truck_assignment():
    start_time = time.time()

    #--------------------------------------------------------
    # 1) 데이터 불러오기
    #--------------------------------------------------------
    input_file = "/content/sample_data/100개운송사_50개운송구간_기준정보A선행결과통합.xlsx"
    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']
    max_trucker_num = int(pd.read_excel(input_file, sheet_name=0, usecols="M").iloc[0, 0])
    max_ratio = float(pd.read_excel(input_file, sheet_name=0, usecols="P").iloc[0, 0])

    # 결측값 제거 및 숫자 변환
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)

    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df_demand.dropna(subset=['필요대수'], inplace=True)

    #--------------------------------------------------------
    # 2) 파라미터 세팅
    #--------------------------------------------------------
    cost_dict = {}      # (i, r, t): 운송단가
    capacity_dict = {}  # (i, r, t): 트럭보유대수
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])
        cost_dict[key] = row['운송단가']
        capacity_dict[key] = row['트럭대수']

    demand_dict = {}    # (r, t): 필요대수
    for _, row in df_demand.iterrows():
        key = (row['운송구간'], row['트럭유형'])
        demand_dict[key] = row['필요대수']

    truckers = df['운송사'].unique()
    route_type_list = demand_dict.keys()

    #--------------------------------------------------------
    # 3) OR-Tools 모델 생성
    #--------------------------------------------------------
    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print("Solver 초기화 실패")
        return

    #--------------------------------------------------------
    # 4) 변수 정의
    #--------------------------------------------------------
    x_vars = {}  # 트럭 수 변수
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = solver.IntVar(0, capacity_dict[(i, r, t)], f"x_{i}_{r}_{t}")

    y_vars = {i: solver.BoolVar(f"y_{i}") for i in truckers}

    #--------------------------------------------------------
    # 5) 목적함수: 총 운송비 최소화
    #--------------------------------------------------------
    solver.Minimize(solver.Sum(cost_dict[(i, r, t)] * x_vars[(i, r, t)] for (i, r, t) in x_vars))

    #--------------------------------------------------------
    # 6) 제약조건 추가
    #--------------------------------------------------------

    # (1) 수요 충족
    for (r, t) in route_type_list:
        solver.Add(solver.Sum(x_vars[(i, r, t)] for i in truckers if (i, r, t) in x_vars) == demand_dict[(r, t)])

    # (2) 트럭 보유량 이내로 배정
    for (i, r, t), cap in capacity_dict.items():
        solver.Add(x_vars[(i, r, t)] <= cap)

    # (3) 운송사 선택 여부에 따라 트럭 배정 가능
    for (i, r, t) in x_vars:
        solver.Add(x_vars[(i, r, t)] <= capacity_dict[(i, r, t)] * y_vars[i])

    # (4) 선택된 운송사 수 제한
    solver.Add(solver.Sum(y_vars[i] for i in truckers) <= max_trucker_num)

    # (5) 운송사당 최대 배정량 제한 (X_MAX 적용)
    for (r, t) in route_type_list:
        demand = demand_dict[(r, t)]
        x_max = math.ceil(demand * max_ratio)
        for i in truckers:
            if (i, r, t) in x_vars:
                solver.Add(x_vars[(i, r, t)] <= x_max)

    #--------------------------------------------------------
    # 7) 최적화 실행
    #--------------------------------------------------------
    status = solver.Solve()

    if status not in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print("=> 유효한 해를 찾지 못했습니다.")
        return

    total_cost = solver.Objective().Value()
    print("총 운송비:", total_cost)

    #--------------------------------------------------------
    # 8) 결과 저장
    #--------------------------------------------------------
    result_data = []
    for (i, r, t), var in x_vars.items():
        val = var.solution_value()
        if val > 0:
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(val)])

    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    df_result.sort_values(by=['운송구간', '트럭유형'], inplace=True)

    output_file = "/content/sample_data/100개운송사_50개운송구간_기준정보A선행결과통합_최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))

    end_time = time.time()
    print("\n프로그램 수행시간(초):", round(end_time - start_time, 4))

if __name__ == "__main__":
    optimize_truck_assignment()


총 운송비: 990450.0

--- 배정 결과 ---
  운송사 운송구간 트럭유형  운송단가  선택된트럭수
운송사59 구간01 트럭01   213       3
운송사74 구간01 트럭01   147      27
운송사41 구간01 트럭02   176      27
운송사46 구간01 트럭02   391       3
운송사54 구간01 트럭03   270       3
운송사58 구간01 트럭03   147      27
운송사46 구간01 트럭04   152       3
운송사62 구간01 트럭04   107      27
운송사58 구간02 트럭01   301       3
운송사62 구간02 트럭01   248      27
운송사41 구간02 트럭02   194      27
운송사74 구간02 트럭02   289       3
운송사41 구간02 트럭03   100      27
운송사74 구간02 트럭03   198       3
운송사44 구간02 트럭04   208       3
운송사46 구간02 트럭04   159      27
운송사51 구간03 트럭01   193       3
운송사59 구간03 트럭01   104      27
운송사44 구간03 트럭02   166      27
운송사46 구간03 트럭02   229       3
운송사42 구간03 트럭03   201       3
운송사74 구간03 트럭03   195      27
운송사54 구간03 트럭04   163       3
운송사74 구간03 트럭04   123      27
운송사42 구간04 트럭01   230       3
운송사54 구간04 트럭01   223      27
운송사41 구간04 트럭02   107      27
운송사44 구간04 트럭02   122       3
운송사42 구간04 트럭03   345       3
운송사54 구간04 트럭03   277      27
운송사42 구간04 트럭04   206       3
운송사59 구간0